Video Games Recommendation System "Hall of Fame"
===

*Team: Solo-Master <br/>
Lead: Vladimir Yu. Ivanov*

<img src="odroid.jpg" alt="drawing" width="512"/>

Source of data:

- https://www.vgchartz.com/ -- video game journalism resource with DB of historical info about sold video games copies
- https://github.com/GregorUT/vgchartzScrape -- ready to use scrapper of the DB from that resource

Data are stored as ***vgsales.csv***

In [1]:
import numpy            as np
import pandas           as pd
import pandas_profiling as pf
import ipywidgets       as widget

from typing import List

from sklearn.metrics.pairwise import cosine_similarity as Cos_d
from sklearn.preprocessing    import OneHotEncoder     as OrthoEncoder

from IPython.display import display
from IPython.display import clear_output

Data Exploring
---

In [2]:
df = pd.read_csv('vgsales.csv')
df.head(n=10)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,6,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
6,7,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
8,9,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.59,7.06,4.70,2.26,28.62
9,10,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31


In [3]:
print(f'total games: {df.shape[0]}')

total games: 16598


Columns description:

- **Rank** - ranking of overall sales
- **Name** - game name
- **Platform** - platform the game was released on (i.e. Gameboy, Playstation)
- **Year** - game release year
- **Genre** - game genre
- **Publisher** - game publisher
- **NA_Sales** - sales in North America (in millions)
- **EU_Sales** - sales in Europe (in millions)
- **JP_Sales** - sales in Japan (in millions)
- **Other_Sales** - sales in the rest of the world (in millions)
- **Global_Sales** - total worldwide sales (in millions)

Quick EDA
---

In [4]:
pf.ProfileReport(df).to_file('profile_1.html')

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

- Platforms: 31
- Years: 1980 - 2020
- Genres: 12
- Publishers: 578
- Sales groups: 4
- and **NaNs**!

In [5]:
df = df.dropna()

In [6]:
pf.ProfileReport(df).to_file('profile_2.html')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

- Platforms: 31
- Years: 1980 - 2020
- Genres: 12
- Publishers: 576
- Sales groups: 4

In [7]:
print(f'total games: {df.shape[0]}')

total games: 16291


Popularity-based System
---

In [11]:
TOP_K = 10

FILTER_TYPE = [
    'Platforms' ,
    'Years'     ,
    'Genres'    ,
    'Publishers',
    'Sales'     ,
]

FILTER_LIST = [
    widget.SelectMultiple(
        options=[
            'DS'  ,
            'PS2' ,
            'PS3' ,
            'Wii' ,
            'X360',
            'PSP' ,
            'PS'  ,
            'PC'  ,
            'XB'  ,
            'GBA' ,
            'NES' ,
        ],
        rows=10+1,
    ),
    widget.IntRangeSlider(
        value=[1980, 2020],
        min=1980          ,
        max=2020          ,
        step=1            ,
        readout_format='d',
    ),
    widget.SelectMultiple(
        options=[
            'Action'      ,
            'Sports'      ,
            'Misc'        ,
            'Role-Playing',
            'Shooter'     ,
            'Adventure'   ,
            'Racing'      ,
            'Platform'    ,
            'Simulation'  ,
            'Fighting'    ,
        ],
        rows=10,
    ),
    widget.SelectMultiple(
        options=[
            'Electronic Arts'             ,
            'Activision'                  ,
            'Namco Bandai Games'          ,
            'Ubisoft'                     ,
            'Konami Digital Entertainment',
            'THQ'                         ,
            'Nintendo'                    ,
            'Sony Computer Entertainment' ,
            'Sega'                        ,
            'Take-Two Interactive'        ,
        ],
        rows=10,
    ),
    widget.RadioButtons(
        options=[
            'NA_Sales'    ,
            'EU_Sales'    ,
            'JP_Sales'    ,
            'Other_Sales' ,
            'Global_Sales',
        ],
        value='Global_Sales',
    ),
]

tabs = widget.Tab()
butn = widget.Button(description='recommend', button_style='success', icon='check')

tabs.children = FILTER_LIST
for (idx, val) in enumerate(FILTER_TYPE):
    tabs.set_title(idx, val)
    
def click_cb(btn):
    '''
    Button click callback: filters source dataset and prints recommendation.
        btn [inp]: button object the callback was triggered on
    '''
    assert btn, 'oops'
    result = df.copy()
    for (idx, itm) in enumerate(FILTER_LIST):
        name = FILTER_TYPE[idx]
        data = itm.value
        if name == 'Platforms' and data:
            result = result[result['Platform'].isin(data)]
        if name == 'Years':
            result = result[(data[0] <= result['Year']) & (result['Year'] <= data[1])]
        if name == 'Genres' and data:
            result = result[result['Genre'].isin(data)]
        if name == 'Publishers' and data:
            result = result[result['Publisher'].isin(data)]
        if name == 'Sales':
            result = result.sort_values(by=data, ascending=False)
    clear_output(), display(tabs), display(butn)
    result = result.head(TOP_K)
    if len(result) > 0:
        display(result.drop(labels=('Rank'), axis='columns'))
    else:
        print('no recommendations due to too restricted filter')

butn.on_click(click_cb)

display(tabs)
display(butn)

Button(button_style='success', description='recommend', icon='check', style=ButtonStyle())

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.59,7.06,4.70,2.26,28.62
9,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31


Content-based System
---

In [12]:
TOP_K = 10

FEATURE_TYPE = [
    'Platform' ,
    'Years'    ,
    'Genre'    ,
    'Publisher',
]

FEATURE_LIST = [
    widget.Dropdown(
        options=[
            ''    ,
            'DS'  ,
            'PS2' ,
            'PS3' ,
            'Wii' ,
            'X360',
            'PSP' ,
            'PS'  ,
            'PC'  ,
            'XB'  ,
            'GBA' ,
            'NES' ,
        ],
    ),
    widget.IntRangeSlider(
        value=[1980, 2020],
        min=1980          ,
        max=2020          ,
        step=1            ,
        readout_format='d',
    ),
    widget.Dropdown(
        options=[
            ''            ,
            'Action'      ,
            'Sports'      ,
            'Misc'        ,
            'Role-Playing',
            'Shooter'     ,
            'Adventure'   ,
            'Racing'      ,
            'Platform'    ,
            'Simulation'  ,
            'Fighting'    ,
        ],
    ),
    widget.Dropdown(
        options=[
            ''                            ,
            'Electronic Arts'             ,
            'Activision'                  ,
            'Namco Bandai Games'          ,
            'Ubisoft'                     ,
            'Konami Digital Entertainment',
            'THQ'                         ,
            'Nintendo'                    ,
            'Sony Computer Entertainment' ,
            'Sega'                        ,
            'Take-Two Interactive'        ,
        ],
    ),
]

tabs = widget.Tab()
butn = widget.Button(description='recommend', button_style='success', icon='check')

tabs.children = FEATURE_LIST
for (idx, val) in enumerate(FEATURE_TYPE):
    tabs.set_title(idx, val)

def click_cb(btn):
    '''
    Button click callback: filters source dataset and prints recommendation.
        btn [inp]: button object the callback was triggered on
    '''
    assert btn, 'oops'
    usergame_to_find = [] # List[str]
    features_to_pick = [] # List[str]
    year_begin = []       # List[int] 
    year_end   = []       # List[int]
    for (idx, itm) in enumerate(FEATURE_LIST):
        name = FEATURE_TYPE[idx]
        data = itm.value
        if name == 'Platform' and data:
            usergame_to_find.append(data)
            features_to_pick.append(name)
        if name == 'Years':
            year_begin = data[0]
            year_end   = data[1]
        if name == 'Genre' and data:
            usergame_to_find.append(data)
            features_to_pick.append(name)
        if name == 'Publisher' and data:
            usergame_to_find.append(data)
            features_to_pick.append(name)
    clear_output(), display(tabs), display(butn)
    if len(features_to_pick) > 0:
        vec = np.array([usergame_to_find])
        mtx = df[features_to_pick].to_numpy()
        enc = OrthoEncoder().fit(mtx)
        vec = enc.transform(vec).toarray()
        mtx = enc.transform(mtx).toarray()
        assert vec.shape[1] == mtx.shape[1]
        scores = Cos_d(mtx, vec).flatten()
        top_id = np.flip(np.argsort(scores))
        rec = df.iloc[top_id][
            ['Name', 
             'Platform', 
             'Year',  
             'Genre', 
             'Publisher']]
        rec.insert(5, 'Cos_d', scores[top_id])
        rec = rec[(year_begin <= rec['Year']) & (rec['Year'] <= year_end)]
        display(rec.head(TOP_K))
    else:
        print('please pick at least one feature')
    
butn.on_click(click_cb)

display(tabs)
display(butn)

Button(button_style='success', description='recommend', icon='check', style=ButtonStyle())

,Name,Platform,Year,Genre,Publisher,Cos_d
7364,World Poker Tour,PS2,2005.0,Misc,Take-Two Interactive,1.000000
11594,Ape Escape: Pumped & Primed,PS2,2004.0,Misc,Sony Computer Entertainment,0.666667
5067,EyeToy Play 2,PS2,2004.0,Misc,Sony Computer Entertainment,0.666667
12266,World Poker Tour,XB,2005.0,Misc,Take-Two Interactive,0.666667
3941,Capcom Classics Collection,PS2,2005.0,Misc,Capcom,0.666667
1032,Rockstar Games Double Pack: Grand Theft Auto I...,PS2,2003.0,Action,Take-Two Interactive,0.666667
7842,Frequency,PS2,2001.0,Misc,Sony Computer Entertainment,0.666667
2764,Jampack Summer 2001,PS2,2001.0,Misc,Sony Computer Entertainment,0.666667
7850,Jampack: Summer 2003 (RP-T),PS2,2003.0,Misc,Sony Computer Entertainment,0.666667
15959,Technic Beat,PS2,2002.0,Misc,Arika,0.666667


Conclusion
---

- Improvisation here